In [145]:
# Qn 1 (10 points)
# Read the file ‘grades.csv’. Write code that finds and displays the 5 student IDs 
# and their corresponding grades who received the lowest grades in the class. 
# Some students have not appeared for the exam, but they are not to be treated as 0. 
# They need to be simply skipped from the computation. (20points)


import zipfile
import csv

with zipfile.ZipFile("C:\\Code\\MFG598\\Homework\\Resources\\HW3_Data_Files.zip", "r") as unzipped_file:
    unzipped_file.extractall()

with open("grades.csv", "r") as csv_file:
    csv_contents = csv.DictReader(csv_file)
    nonzero_dict = {}

    for row in csv_contents:
        if row["Scores"]:
            nonzero_dict[row['Student IDs']] = int(row['Scores'])

    sorted_nonzero_dict = dict(sorted(nonzero_dict.items(), key=lambda item: item[1]))

    sorted_nonzero_dict_lowest_five = list(sorted_nonzero_dict.items())[:5]

    print(f"The lowest 5 non-zero scores and their associated values are: \n {sorted_nonzero_dict_lowest_five}")

The lowest 5 non-zero scores and their associated values are: 
 [('103', 56), ('109', 66), ('106', 75), ('104', 78), ('113', 86)]


In [144]:
# Qn 2 (20 points)
# Open and read the contents of the file – ‘2_NoofParts_assem.txt’. Perform the following (20 points):
# a)	Calculate how many entries are available in that file excluding the header.
# b)	Calculate the sum of all parts from each file. Essentially, finding the sum of all values contained in the 2nd column of the file.
# c)	Extract the part ID that has the largest associated no. of parts from the entire list_input.


file = "2_NoofParts_assem.txt"

import zipfile
import chardet

with zipfile.ZipFile("C:\Code\MFG598\Homework\Resources\HW3_Data_Files.zip", 'r') as unzipped:
    unzipped.extractall()

#When I try to open here, I get a decode error for utf-8. What type of encoding is this file?

#"rb" does "read binary" so we can look deeply in the file.
with open("2_NoofParts_assem.txt", "rb") as file:
    result = chardet.detect(file.read())
    print(result['encoding'])

#Not sure how to integrate this into the encoding = statement below. I'm sure there's a way.

with open("2_NoofParts_assem.txt", 'r', encoding = 'utf-16') as file:
    #print(file.read())
    lines = file.readlines()
    num_lines = len(lines)

print(f'There are {num_lines - 1} lines in the file, excluding the header')

#Doing my EDA here
#print(lines) 
#Based on this print statement, our values are separated by \t and followed by \t\t\n
#There's a value to 'clean' the statement but I don't remember right now.

#Let's first split the items.

split_values = []
for item in lines:
    split_values.append(item.split("\t"))

#print(split_values)

#From this print statement, we see that the header is still included. We can take it out now or later, because it will interfere with int operations.
#We can nest the entire value-stripping code block in a 'try' statement and then exit if we have a ValueError.
#We also need to account for the fact that the list_input now consists of lists.

values_only = []
for sublist in split_values:
    for item in sublist:
        try:
            values_only.append(int(item))
        except ValueError:
            continue
#print(values_only)

print(f"The sum of all integer values in the .txt file is {sum(values_only)}")

# c)	Extract the part ID that has the largest associated no. of parts from the entire list_input.

#We need to know which entry in the list_input of lists has the max value, then we can just navigate to it.
#My instinct is to iterate through the list_input increasing an index number, but I feel like there's a more elegant way.
#I learned about custom keys and will apply that here.

#This definition gives us the key value of whatever term in the master list_input that we want.
def extract_max_num(item):
    return (item[1])

#This item finds the sublist. The value in the brackets lets us 'skip' the header, starting at 1 and going onward.
max_part = max(split_values[1:], key=extract_max_num)

#The definition acts like a lambda function in that the value of max() is used as a parameter.

part_id = max_part[0]

print(f"The part ID with the highest # of associated parts is {part_id}")
print(f"The number of associated parts for that ID is {max_part[1]}")

UTF-16
There are 1340 lines in the file, excluding the header
The sum of all integer values in the .txt file is 161509
The part ID with the highest # of associated parts is 122f3850-6378-489c-ae21-b88c035c2818
The number of associated parts for that ID is 99


In [143]:
# Qn 3 (25 points)
# Using a python script, open the files contained in the .zip file – “3_Jobs_Completed_log.zip”. 
# Scan the files for the line that starts with the word string – “Jobs Completed.. ”. Extract the number associated with this line and for all instances that this word string appears across all log files, count the total sum across all files contained within the .zip file. (20points) 

# For example:

#  	Jobs Completed.. 10 2018-09-04 08:21:28.503153

# Extract the number 10 from this sentence which signifies the total number of jobs completed at the point in time.  
# Find for all instances in which the string - ‘Jobs Completed’ appears, 
# find the total number of jobs completed by adding the numbers from across the provided log files.

import zipfile
import chardet

filetypes = []
total_sum = 0
#Opening .zip within .zip here and extracting:
with zipfile.ZipFile("C:\Code\MFG598\Homework\Resources\HW3_Data_Files.zip", 'r') as wrapper:
    with wrapper.open("Jobs_Completed_log.zip", 'r') as Jobs_Completed_file:
            with zipfile.ZipFile(Jobs_Completed_file) as Jobs_Completed_log:

#Next I'll iterate through a list_input of the files and detect encoding for each with chardet, since I have no idea if they are homoegenous.
#Is this overkill?
#To make the code resilient, I'll pass the encoding data to another 

                #I was doing this by habit, but it looks like it's not necessary if I work within the zipfile condition. 
                #   Not sure if this is preferable.
                # Jobs_Completed_log.extractall()

                Jobs_Completed_log_file_list = Jobs_Completed_log.namelist()
                for file in Jobs_Completed_log_file_list[:len(Jobs_Completed_log_file_list)]:
                    with Jobs_Completed_log.open(file) as f:
                        file_data = f.read()
                        encoding_result = chardet.detect(file_data)
                        encoding = encoding_result["encoding"]
                        filetypes.append(encoding)

                        #Found that a typical str.split does not work here like splitting a utf-8 file does.
                            # Resulted in a TypeError 
                            # Instead, I'll 'decode'.
                        file_data_str = file_data.decode(encoding)
                        lines = file_data_str.split("\n")

                        #Now that we have useful data, I'll search for the Jobs Completed lines, 
                        #   split then up, and grab the total number with an index.
                        for line in lines:
                            if line.startswith("Jobs Completed.. "):
                                parts = line.split(" ")
                                total_sum += int(parts[2])

#print("Filetypes:", filetypes)
print("Jobs completed:", total_sum)
#print(file_data)



Jobs completed: 2384


In [ ]:
# Qn 4 (20 points)
# For the same files above, calculate the following:
# a)	How many jobs were completed in the period between 15th Aug 2018 to 15th Sept 2018?
# b)	The day of the year in which the maximum number of jobs was completed.
# c)	Calculate the total number of days elapsed between the time at which the first job batch was completed to the last batch completed.

import zipfile
import chardet
from datetime import datetime, timedelta
import re

#Rewriting this part to help me remember it.
filetypes = []
encoding = []
date_list = []

with zipfile.ZipFile("C:\Code\MFG598\Homework\Resources\HW3_Data_Files.zip", "r") as unzipped_outer:
    with unzipped_outer.open("Jobs_Completed_log.zip", "r") as inner_container:
        with zipfile.ZipFile(inner_container, "r") as unzipped_inner:

            file_list = unzipped_inner.namelist()
            for file in file_list[:len(file_list)]:
                with unzipped_inner.open(file) as f:
                    file_data = f.read()
                    #print(file_data)
                    encoding_result = chardet.detect(file_data)
                    encoding = encoding_result["encoding"]
                    #print(encoding)
                    filetypes.append(encoding)

#I got a bit distracted with this common_encoding block. I was trying to figure out how to make the code filetype agnostic. 
#Upon reflection, I may have been able to nest chardet.detect as such: file_data.decode(chardet.detect(file_data))

def common_encoding(list_input):
    if all(x == list_input[0] for x in list_input):
        return True
    else:
        return False

while True:
    if common_encoding(filetypes):
        date_list.append(re.findall(r'\b\d{4}-\d{2}-\d{2}', file_data.decode(filetypes[0])))       
        break


# define start and end date of the range
start_date = datetime(2018, 8, 15)

end_date = datetime(2018, 9, 15)

with zipfile.ZipFile("C:\Code\MFG598\Homework\Resources\HW3_Data_Files.zip", "r") as unzipped_outer:
    with unzipped_outer.open("Jobs_Completed_log.zip", "r") as inner_container:
        with zipfile.ZipFile(inner_container, "r") as unzipped_inner:
        # Loop over the 5 text files inside the inner .zip file
            for filename in unzipped_inner.namelist():
                if filename.endswith('.txt'):
                    # Read the text file
                    with unzipped_inner.open(filename) as txt_file:
                        txt_data = txt_file.read()
                        
                        data_split = txt_data.split(b'Date:')
                        if len(data_split) > 1:
                            date_str = data_split[1].strip()

                            date_list.append(date_str)                        
                        # Add the parsed date to the list of parsed dates
#print(date_list)

parsed_dates = [datetime.strptime(date, '%Y-%m-%d') for date in str(date_list)]

count = 0
for date in parsed_dates:
    if start_date <= date <= end_date:
        count += 1



In [146]:
# Qn 5 (25 points)
# Read the following (6_Part1.stl) triangular mesh file available in ASCII format. 
# For those of you aware of 3D printing, you will recognize this is as a .STL file. 
# For those who aren’t aware of what a .STL file is, please read up online on the format of a .STL ascii file. 
# An ASCII formatted .STL file can be opened in any text editor. 
# Every vertex of the triangle is preceeded by the word ‘vertex’ followed by the x, y and z coordinates. 
# Therefore, ONE Triangle should have 3 vertex entries. Typically, the units is not specified, 
# but you can assume it to be in millimeters. 

# Write functions that calculate the following: (20 points)
# a.	Find the total number of triangles listed in the file.
# b.	Store the coordinates of each triangle in a 3-tuple list_input containing N indices, where N is the total number of triangles.
# c.	list_input the area of each triangle in the file. Compute the total surface area of all triangles listed in the file. Read up online on the formula to calculate the area of a triangle, given three vertices.
# '''

file = "C:/Code/MFG598/Homework/6_Part1.stl"
vertex_count = 0

#Using the counting function is going to be more lightweight than using file.readlines()
    #...then iterating through the list of lines. We'll readlines later however.
with open(file, "r") as stl_file:
    file_data = stl_file.read()
    vertex_count = file_data.count("vertex")

triangle_count = vertex_count/3

print(f"The number of triangles is: {int(triangle_count)}")

#Now we're using readlines and we'll use 'vertex' as a pointer to access just the tuple of values after it.
with open(file, "r") as file:
    file_data = file.readlines()

triangles = []
for line in file_data:
    if 'vertex' in line:
        coordinates = tuple([float(x) for x in line.strip().split()[1:]])
        triangles.append(coordinates)

#Now find the total area of all triangles, and also the area of each triangle.
#I'm not sure if the question wants me to append this to list_input

area_list = []


tri = [triangles[i:i+3] for i in range(0, len(triangles), 3)]

#Going to assume that the triangle format is Ax, Ay, Az, Bx, By, Bz...
#Iterate through each grouping of vertices and then do the math for volume.
def triangle_volume(tri):
    x1, y1, z1 = tri[0]
    x2, y2, z2 = tri[1]
    x3, y3, z3 = tri[2]
    return abs(x1*y2*z3 + x2*y3*z1 + x3*y1*z2 - x1*y3*z2 - x2*y1*z3 - x3*y2*z1) / 6

volumes = [triangle_volume(triangle) for triangle in tri]
print(f"Total area of all triangles is: {sum(volumes)}. \n The areas of each triangle in the mesh follow:")
print(volumes)


The number of triangles is: 44
Total area of all triangles is: 1022629.4450895642. 
 The areas of each triangle in the mesh follow:
[0.0, 0.0, 0.0, 2638.9353534853403, 5472.944843684953, 13085.550675119106, 9744.057557894803, 0.0, 106497.19095809023, 15631.535308557655, 145880.1784239923, 145880.1784239923, 106497.19095809023, 9459.949236665683, 9459.949236665685, 28294.53112314489, 3692.9550878294663, 28294.5311231449, 26323.460256115435, 3692.9550878294663, 45158.04214259463, 45158.04214259464, 26323.460256115435, 5472.944843684953, 5472.944843684953, 2638.9353534853403, 0.0, 9933.88744898081, 2638.9353534853412, 0.0, 6537.786042777389, 9933.887448980808, 7099.877958781198, 7099.877958781198, 3566.0235234441257, 13167.256416249083, 33228.97134803277, 11531.591827811746, 28377.47583342216, 27533.410225028416, 15783.804821429241, 11188.593735626715, 14246.033970931187, 29991.567939339788]
